In [16]:
import pandas as pd
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [17]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,hauteur_d_entrejambe,...,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras
1747,190.7032,55.0,101.999233,1.0,29.4894,67.0052,112.2934,NaN,137.3632,85.2932,...,52.7939,51.5874,43.9166,157.2895,52.7050,62.3443,63.0936,98.1964,117.9068,26.8605
1444,177.8000,30.0,75.500388,1.0,26.1874,63.1952,99.5934,NaN,127.6604,80.7974,...,48.8061,48.8950,42.4688,146.6469,47.0916,58.0136,54.4068,82.9056,105.9942,25.7429
3161,181.8894,41.0,90.945196,1.0,28.2956,66.5988,108.8898,NaN,131.8006,80.1116,...,49.6062,48.4124,44.6278,149.3012,51.4096,57.6199,57.9882,92.2020,105.5116,26.2636
2283,179.4002,47.0,108.635284,1.0,27.9908,64.0080,115.4938,NaN,131.1910,78.5114,...,51.0032,50.2920,42.1640,149.6060,55.0926,59.8678,66.3956,107.7976,104.5972,27.1907
3635,180.5940,20.6,64.600573,1.0,25.1968,63.2968,91.7956,NaN,130.4036,83.4898,...,49.3014,47.0916,39.1922,146.5072,44.0944,60.2996,53.0098,76.2000,103.9876,25.9715


In [18]:
caesar_male["tour_de_cuisse_ratio"] = caesar_male["tour_de_cuisse"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [19]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if tour_cuisse < 0.312557:
        cuisses = "fines"
    elif  0.312557 <=tour_cuisse<0.350709 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [20]:
# 1. Wrapper
class MultiTPOTWrapper:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        import numpy as np
        predictions = [m.predict(X) for m in self.models]
        return np.vstack(predictions).T

In [21]:
# 2. Pré-traitement
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_male[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [22]:
# 3. Chargement modèles
import joblib
import os

dossier_pipelines = "pipelines_complets"
models = []
for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        models.append(joblib.load(path))

In [ ]:
import joblib
import os

dossier_pipelines = "pipelines_complets"
models = []

for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        model = joblib.load(path)
        models.append(model)
        print(f"✅ Modèle chargé : {filename}")

In [23]:
# 4. Pipeline complet
full_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("multi_model", MultiTPOTWrapper(models))
])

In [27]:
# 5. Sauvegarde
joblib.dump(full_pipeline, "full_pipeline.pkl")

# 6. Prédiction
pipeline_loaded = joblib.load("full_pipeline.pkl")

In [11]:
# ✅ Créer un wrapper à partir des pipelines chargés
wrapper = MultiTPOTWrapper(models)
new_data = pd.DataFrame([{
    "taille": 187,
    "age": 33,
    "weight": 80,
    "categorie_ventre": "moyen",
    "categorie_torse": "large",
    "categorie_cuisses": "medium"
    
}])
# ✅ Prédictions
preds = wrapper.predict(new_data)


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.

In [12]:
# Suppose que tu as déjà ton pipeline chargé
# pipeline_loaded est de type Pipeline(steps=[("preprocess", ...), ("multi_model", ...)])

# Accès au préprocesseur uniquement
preprocessor = pipeline_loaded.named_steps["preprocess"]

# Applique le transformateur au new_data brut
X = preprocessor.transform(new_data)

# Si tu veux voir sous forme de DataFrame
import pandas as pd

# Récupère les noms des colonnes après transformation
# Cela fonctionne uniquement si le ColumnTransformer est simple
try:
    feature_names = preprocessor.get_feature_names_out()
    X_df = pd.DataFrame(X, columns=feature_names)
except:
    # fallback si les noms ne sont pas récupérables
    X_df = pd.DataFrame(X)

# Affiche les premières lignes
X_df.head()
X_df.columns

Index(['num__taille', 'num__age', 'num__weight', 'cat__categorie_ventre_moyen',
       'cat__categorie_ventre_plat', 'cat__categorie_ventre_rond',
       'cat__categorie_torse_fin', 'cat__categorie_torse_large',
       'cat__categorie_torse_moyen', 'cat__categorie_cuisses_fines',
       'cat__categorie_cuisses_larges', 'cat__categorie_cuisses_moyennes'],
      dtype='object')

In [25]:
# ✅ Extraire les noms des cibles
noms_variables = [f.replace("pipeline_tpot_", "").replace(".pkl", "") 
                  for f in sorted(os.listdir(dossier_pipelines)) if f.endswith(".pkl")]

# ✅ Associer les valeurs
predictions_dict = dict(zip(noms_variables, preds[0]))

# ✅ Affichage
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


📊 Prédictions :
 - hauteur_d_entrejambe : 86.02
 - hauteur_de_la_taille : 110.91
 - hauteur_de_poitrine : 134.75
 - hauteur_des_epaules : 151.35
 - hauteur_des_genoux : 52.07
 - hauteur_des_hanches : 93.48
 - largeur_d_epaule : 43.45
 - largeur_des_epaules_a_l_horizontales : 43.20
 - longueur_d_avant_bras : 10.89
 - longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 60.93
 - longueur_du_bras : 66.94
 - tour_de_cheville : 27.35
 - tour_de_cuisse : 59.78
 - tour_de_poitrine : 96.97
 - tour_de_taille : 82.57
 - tour_du_cou : 41.31


In [14]:
import os
import joblib
import numpy as np
from sklearn.preprocessing import RobustScaler

# 📁 Dossier scalers
scalers_folder = "scalers"
os.makedirs(scalers_folder, exist_ok=True)

# Exemple : predictions_dict = {'hauteur_d_entrejambe': 0.83, 'hauteur_de_la_taille': 0.91, ...}

# Enregistrer un scaler pour chaque variable
for var, val in predictions_dict.items():
    scaler = RobustScaler()
    # On fit le scaler sur la valeur (simulateur)
    X = np.array([[val]])
    try:
        scaler.fit(X)
        scaler_path = os.path.join(scalers_folder, f"scaler_{var}.pkl")
        joblib.dump(scaler, scaler_path)
        print(f"✅ Scaler sauvegardé pour {var}")
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde du scaler pour {var} : {e}")

# Affichage des prédictions
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


✅ Scaler sauvegardé pour hauteur_d_entrejambe
✅ Scaler sauvegardé pour hauteur_de_la_taille
✅ Scaler sauvegardé pour hauteur_de_poitrine
✅ Scaler sauvegardé pour hauteur_des_epaules
✅ Scaler sauvegardé pour hauteur_des_genoux
✅ Scaler sauvegardé pour hauteur_des_hanches
✅ Scaler sauvegardé pour largeur_d_epaule
✅ Scaler sauvegardé pour largeur_des_epaules_a_l_horizontales
✅ Scaler sauvegardé pour longueur_d_avant_bras
✅ Scaler sauvegardé pour longueur_de_la_colonne_vertebrale_jusqu_au_poignet
✅ Scaler sauvegardé pour longueur_du_bras
✅ Scaler sauvegardé pour tour_de_cheville
✅ Scaler sauvegardé pour tour_de_cuisse
✅ Scaler sauvegardé pour tour_de_poitrine
✅ Scaler sauvegardé pour tour_de_taille
✅ Scaler sauvegardé pour tour_du_cou
📊 Prédictions :
 - hauteur_d_entrejambe : 86.02
 - hauteur_de_la_taille : 110.91
 - hauteur_de_poitrine : 134.75
 - hauteur_des_epaules : 151.35
 - hauteur_des_genoux : 52.07
 - hauteur_des_hanches : 93.48
 - largeur_d_epaule : 43.45
 - largeur_des_epaules_a_

In [15]:
import joblib
import os
import numpy as np

def descale_predictions_robust(predictions_dict, dossier_scalers="scalers_y"):
    predictions_descalees = {}
    for nom, valeur in predictions_dict.items():
        if nom == "longueur_du_bras":
            # Cette variable est déjà dans les bonnes unités
            predictions_descalees[nom] = valeur
        else:
            path_scaler = os.path.join(dossier_scalers, f"scaler_{nom}.pkl")
            if os.path.exists(path_scaler):
                scaler = joblib.load(path_scaler)
                valeur_realiste = scaler.inverse_transform([[valeur]])[0, 0]
                predictions_descalees[nom] = round(valeur_realiste, 2)
            else:
                print(f"⚠️ Scaler manquant pour : {nom}")
                predictions_descalees[nom] = valeur  # au cas où on veut garder la valeur
    return predictions_descalees
    

# Déscalage
predictions_descalees = descale_predictions_robust(predictions_dict)

# Affichage des vraies valeurs
print("\n✅ Prédictions déscalées :")
for k, v in predictions_descalees.items():
    print(f"{k} : {v}")



✅ Prédictions déscalées :
hauteur_d_entrejambe : 723.27
hauteur_de_la_taille : 968.88
hauteur_de_poitrine : 1300.95
hauteur_des_epaules : 1584.59
hauteur_des_genoux : 257.48
hauteur_des_hanches : 797.96
largeur_d_epaule : 177.24
largeur_des_epaules_a_l_horizontales : 244.3
longueur_d_avant_bras : 19.2
longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 310.24
longueur_du_bras : 66.93718604580963
tour_de_cheville : 81.46
tour_de_cuisse : 477.81
tour_de_poitrine : 1383.65
tour_de_taille : 1204.1
tour_du_cou : 204.57
